## Modelos y Simulación – Trabajo Práctico 

## Modelización de Lenguaje Natural con CFG y SDT (Clase – Modelos y Simulación)

**Autor: Cátedra – Modelos y Simulación - Facultad de Ingeniería - UCA**     
**Fecha: 2025-15-09**   
**Documento: MyS_C4A_Modelado Formal con Gramaticas _R2**   
### Temas: **Análisis Léxico**, **Análisis Sintáctico (CFG)**, **Análisis Semántico** y **Traducción Dirigida por la Sintaxis (SDT)**.  
####  Contexto: 
- Ejercitar el proceso del procesamiento de lenguaje: *tokens → parse (CFG) → semántica (SDT)*.
- Implementar un **compositor semántico** paso a paso (λ-cálculo simple) para una oración transitiva.
- Validar **tipos semánticos** y restricciones de selección.
- Construir un **chatbot** de dominio acotado con **CFG + SDT**.
- Explorar un **parser NLTK** sobre una gramática mínima.


```
texto
  └─► Análisis Léxico (tokenización, normalización)
        └─► Análisis Sintáctico (CFG) ─► Árbol / derivación
              └─► Análisis Semántico (SDT, λ-cálculo) ─► Fórmula lógica / AST
                    └─► Motor de Respuestas (reglas, plantillas, acciones)
                          └─► Salida (texto, acción)
```


### Ejemplo de chatbot simple con reglas
#### Lexer con Flex — chatbot.l
#### Parser con Bison - chatbot.y
- Convierte texto a tokens.
- Normaliza: ignoramos mayúsculas con caseless. (Acentos: asumimos que ya vienen sin acentos como parte del preprocesamiento.)
- Cómo ejecutar desde linux:
  - flex -o lexer.c chatbot.l
  - bison -d -o parser.c chatbot.y
  - gcc -o chatbot parser.c lexer.c -lfl
  - ./chatbot


In [ ]:
/* chatbot.y */
%{
  #include <stdio.h>
  #include <stdlib.h>

  /* Prototipos provistos por Flex */
  int yylex(void);
  void yyerror(const char *s);

%}

/* Pedimos a Bison que genere el header con tokens (parser.h) usando -d */
%define api.value.type {int}

/* Declaración de tokens (deben coincidir con los que retorna Flex) */
%token HOLA BUENOS_DIAS HOL BUENAS_NOCHES
%token QUE HORA ES
%token QUIERO INFORMACION SOBRE CURSOS
%token EOL OTHER

/* Precedencias no necesarias aquí, pero podrían usarse en gramáticas más complejas */

%%

/*  Gramática de alto nivel  */

/* Permite múltiples líneas (comandos) */
input
  : /* vacío */
  | input linea
  ;

linea
  : SALUDO EOL      { puts("Chatbot: ¡Hola! ¿Cómo estás?"); }
  | PREGUNTA EOL    { puts("Chatbot: No tengo reloj, pero puedo ayudarte con cursos :)"); }
  | SOLICITUD EOL   { puts("Chatbot: Tenemos cursos de Machine Learning, Simulación y Redes. ¿Cuál te interesa?"); }
  | basura EOL      { puts("Chatbot: Disculpa, no entendí. ¿Podés reformular?"); }
  | EOL             { /* línea vacía: ignorar */ }
  ;

SALUDO
  : HOLA
  | BUENOS_DIAS
  | BUENAS_NOCHES
  ;

PREGUNTA
  : QUE HORA ES
  ;

SOLICITUD
  : QUIERO INFORMACION SOBRE CURSOS
  ;

/* Si la línea contiene tokens desconocidos o estructura no reconocida */
basura
  : basura OTHER
  | OTHER
  ;

%%

/*  Código en C embebido: main, yyerror  */

int main(void) {
  /* yyparse consume tokens de yylex() (Flex) y dispara acciones */
  if (yyparse() == 0) {
    /* parseo exitoso */
    return EXIT_SUCCESS;
  } else {
    return EXIT_FAILURE;
  }
}

void yyerror(const char *s) {
  /* Mensajes de error sintáctico */
  fprintf(stderr, "Parser error: %s\n", s);
}


- En el contexto de chatbots, NLP e IA conversacional, un intent (intención) es la meta o propósito del usuario al escribir un mensaje. Es lo que quiere lograr la persona, más allá de las palabras exactas que usa.
- El uuario escribe:
  - "hola", "buen día" → Intent: SALUDO
  - "qué hora es", "me decís la hora?" → Intent: PREGUNTA_HORA
  - "quiero información sobre cursos", "necesito capacitaciones" → Intent: SOLICITUD_INFO
- El sistema mapea frases distintas al mismo intent.

In [2]:
%%writefile chatbot_ply.py
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Chatbot con PLY (Python Lex-Yacc)
    --
Intents soportados:
- SALUDO: "hola", "buenas", "buenos dias"
- PREGUNTA: "que hora es", "me decis la hora", "tenes la hora", etc.
- SOLICITUD: "quiero informacion sobre cursos [sobre TOPIC]" y variantes

Ejecutar con:
  pip install ply
  python chatbot_ply.py
"""

import sys
import re

#    Dependencia   
try:
    import ply.lex as lex
    import ply.yacc as yacc
except Exception:
    sys.stderr.write("Necesitás instalar PLY primero: pip install ply\n")
    raise

 
# 1) Lexer (tokens, sinónimos, slot)
 

# Lista de tokens que usará el parser
tokens = (
    'HOLA',
    'QUEHORA',
    'QUIERO',
    'INFORMACION',
    'SOBRE',
    'CURSOS',
    'TOPIC',
    'EOL',
)

# Ignorar espacios y tabs
t_ignore = ' \t\r'

# Ignorar puntuación (ruido)
def t_PUNCT(t):
    r'[\,\.\!\?\;\:\(\)\[\]\{\}\-]+'
    pass

# Fin de línea -> EOL
def t_EOL(t):
    r'\n+'
    return t

# Frases para “qué hora es” → un único token (tolerante a variantes)
def t_QUEHORA(t):
    r'(que\s+hora\s+es)|(me\s+dec[ií]s\s+la\s+hora)|(ten[eé]s\s+la\s+hora)|(tienes\s+la\s+hora)|(^|\s)hora(\s|$)'
    t.value = 'que_hora_es'
    return t

# Saludos
def t_HOLA(t):
    r'(hola)|(buenas(\s+(tardes|noches))?)|(buenos\s+d[ií]as)'
    t.value = 'hola'
    return t

# Verbos de intención
def t_QUIERO(t):
    r'(quiero)|(quisiera)|(me\s+gustar[ií]a)'
    t.value = 'quiero'
    return t

# Info / data
def t_INFORMACION(t):
    r'(informaci[oó]n)|(info)|(data)|(detalles)'
    t.value = 'informacion'
    return t

# Preposición/conector
def t_SOBRE(t):
    r'(sobre)|(acerca\s+de)|(de)'
    t.value = 'sobre'
    return t

# Cursos y sinónimos
def t_CURSOS(t):
    r'(curso|cursos|capacitaci[oó]n|capacitaciones|formaci[oó]n)'
    t.value = 'cursos'
    return t

# TOPIC libre (palabras con espacios luego serán tomadas por la gramática)
def t_TOPIC(t):
    r'[A-Za-zÁÉÍÓÚÜÑáéíóúüñ0-9][A-Za-zÁÉÍÓÚÜÑáéíóúüñ0-9\ \-\_]*'
    # Nota: el parser lo usa solo después de "SOBRE"
    t.value = t.value.strip()
    return t

# Errores léxicos: avanzar sin trabar
def t_error(t):
    t.lexer.skip(1)

# Compilar lexer (case-insensitive)
lexer = lex.lex(reflags=re.IGNORECASE | re.UNICODE)

 
# 2) Parser (gramática PLY)
 

# No necesitamos precedencia aquí
precedence = ()

# Entrada: una o más líneas
def p_input(p):
    '''input : input linea
             | linea'''
    pass

def p_linea(p):
    '''linea : saludo EOL
             | pregunta_hora EOL
             | solicitud_cursos EOL
             | EOL'''
    # Acciones implementadas en subreglas

def p_saludo(p):
    'saludo : HOLA'
    print("Chatbot: ¡Hola! ¿Cómo estás?")

def p_pregunta_hora(p):
    'pregunta_hora : QUEHORA'
    print("Chatbot: No tengo reloj, pero puedo ayudarte con cursos :)")

# Variantes de solicitud:
# 1) QUIERO (INFORMACION)? (SOBRE)? CURSOS (SOBRE TOPIC)?
# 2) QUIERO CURSOS (SOBRE TOPIC)?
# 3) QUIERO INFORMACION (SOBRE TOPIC)?
def p_solicitud_cursos(p):
    '''solicitud_cursos : QUIERO opt_info opt_sobre CURSOS opt_tema
                        | QUIERO CURSOS opt_tema
                        | QUIERO INFORMACION opt_tema'''
    if len(p) == 6:
        topic = p[5]
    elif len(p) == 4:
        topic = p[3]
    else:
        topic = None

    if topic:
        print(f'Chatbot: Tenemos cursos de ML, Simulación y Redes sobre "{topic}". ¿Cuál te interesa?')
    else:
        print('Chatbot: Tenemos cursos de ML, Simulación y Redes. ¿Cuál te interesa?')

def p_opt_info(p):
    '''opt_info : INFORMACION
                | empty'''
    pass

def p_opt_sobre(p):
    '''opt_sobre : SOBRE
                 | empty'''
    pass

def p_opt_tema(p):
    '''opt_tema : empty
                | SOBRE tema'''
    if len(p) == 3:
        p[0] = p[2]
    else:
        p[0] = None

def p_tema(p):
    'tema : TOPIC'
    p[0] = p[1]

def p_empty(p):
    'empty :'
    pass

# Recuperación de errores: consumir hasta EOL
def p_error(p):
    if p is None:
        return
    tok = p
    while tok and tok.type != 'EOL':
        tok = parser.token()
    print("Chatbot: Disculpa, no entendí. ¿Podés reformular?")

# Construir parser
parser = yacc.yacc(start='input', debug=False, write_tables=False)

 
# 3) REPL (bucle interactivo)
 
BANNER = (
    "Chatbot PLY — escribí una línea y Enter (Ctrl+D/Ctrl+Z para salir)\n"
    "Ejemplos: 'hola', 'que hora es', 'quiero informacion sobre cursos', "
    "'quisiera cursos sobre deep learning'\n"
)

def main():
    print(BANNER)
    buffer = ""
    try:
        for line in sys.stdin:
            buffer += line
            if not buffer.endswith("\n"):
                continue
            parser.parse(buffer, lexer=lexer)
            buffer = ""
    except KeyboardInterrupt:
        pass

if __name__ == "__main__":
    main()


Overwriting chatbot_ply.py


## 2. Gramática Libre de Contexto (CFG)

Usaremos una **CFG mínima** para oraciones transitivas con objeto introducido por preposición **a**:

Producciones:
```
S   → NP VP
VP  → V NP_a
NP  → Det N
NP_a→ 'a' NP
Det → 'el'
N   → 'gato' | 'perro'
V   → 'muerde'
```


## 3. SDT (Traducción Dirigida por la Sintaxis) con λ-cálculo simple

Asociamos a cada símbolo un atributo semántico `.sem`. Léxico y reglas:

**Léxico (semántica):**
- `N.gato.sem   gato1`
- `N.perro.sem   perro1`
- `Det.el.sem   λx.x`  (definido)
- `V.muerde.sem   λobj.λsubj. MORDER(subj, obj)`

**Reglas (composición):**
- `NP  → Det N`        ⇒  `NP.sem   Det.sem(N.sem)`
- `NP_a→ 'a' NP`       ⇒  `NP_a.sem   NP.sem`  (la preposición no cambia la entidad en esta demo)
- `VP  → V NP_a`       ⇒  `VP.sem   λsubj. V.sem(NP_a.sem)(subj)`
  - El resultado es una función que, dado un sujeto, devuelve una proposición. “aplico el verbo al objeto para obtener un VP que es un predicado de sujetos”
- `S   → NP VP`        ⇒  `S.sem   VP.sem(NP.sem)`

Ejemplo objetivo: **"el gato muerde al perro"** → `MORDER(gato1, perro1)`


## 4. Implementación del compositor semántico.

- Implementamos las acciones semánticas como funciones Python y un *parser manual* que asume la estructura esperada.
- Chatbot basado en CFG + SDT (dominio acotado)¶
- Usamos una clasificación por patrones equivalente a una gramática mínima y asociamos acciones semánticas (respuestas) a cada intención.


In [4]:
%%writefile chatbotCFG_ply.py
# -*- coding: utf-8 -*-
# PLY chatbot CFG 

import re, sys
import ply.lex as lex
import ply.yacc as yacc
import ply.lex as _lexmod

#   LEXER (clase)  
class ChatLexer:
    tokens = (
        'HOLA','QUEHORA','QUIERO','INFORMACION','SOBRE','CURSOS','TOPIC','EOL'
    )
    t_ignore = ' \t\r'

    def t_PUNCT(self, t):
        r'[\,\.\!\?\;\:\(\)\[\]\{\}\-]+'
        pass

    def t_EOL(self, t):
        r'\n+'
        return t

    def t_QUEHORA(self, t):
        r'(que\s+hora\s+es)|(me\s+dec[ií]s\s+la\s+hora)|(ten[eé]s\s+la\s+hora)|(tienes\s+la\s+hora)|(^|\s)hora(\s|$)'
        t.value = 'que_hora_es'
        return t

    def t_HOLA(self, t):
        r'(hola)|(buenas(\s+(tardes|noches))?)|(buenos\s+d[ií]as)'
        t.value = 'hola'
        return t

    def t_QUIERO(self, t):
        r'(quiero)|(quisiera)|(me\s+gustar[ií]a)'
        t.value = 'quiero'
        return t

    def t_INFORMACION(self, t):
        r'(informaci[oó]n)|(info)|(data)|(detalles)'
        t.value = 'informacion'
        return t

    def t_SOBRE(self, t):
        r'(sobre)|(acerca\s+de)|(de)'
        t.value = 'sobre'
        return t

    def t_CURSOS(self, t):
        r'(curso|cursos|capacitaci[oó]n|capacitaciones|formaci[oó]n)'
        t.value = 'cursos'
        return t

    def t_TOPIC(self, t):
        r'[A-Za-zÁÉÍÓÚÜÑáéíóúüñ0-9][A-Za-zÁÉÍÓÚÜÑáéíóúüñ0-9\ \-\_]*'
        t.value = t.value.strip()
        return t

    def t_error(self, t):
        t.lexer.skip(1)

# construir lexer y exponer _lex para helpers
_lex = ChatLexer()
lexer = lex.lex(module=_lex, reflags=re.IGNORECASE | re.UNICODE)

#   PARSER (clase)  
class ChatParser:
    tokens = ChatLexer.tokens
    precedence = ()

    def __init__(self):
        self.last_sem = None

    def p_input(self, p):
        '''input : input linea
                 | linea'''
        pass

    def p_linea(self, p):
        '''linea : saludo EOL
                 | pregunta EOL
                 | solicitud EOL
                 | EOL'''
        pass

    def p_saludo(self, p):
        'saludo : HOLA'
        self.last_sem = {'intent': 'SALUDO'}

    def p_pregunta(self, p):
        'pregunta : QUEHORA'
        self.last_sem = {'intent': 'PREGUNTA'}

    # Variantes de solicitud:
    # 1) QUIERO (INFORMACION)? (SOBRE)? CURSOS (SOBRE TOPIC)?
    # 2) QUIERO CURSOS (SOBRE TOPIC)?
    # 3) QUIERO INFORMACION (SOBRE TOPIC)?
    def p_solicitud(self, p):
        '''solicitud : QUIERO opt_info opt_sobre CURSOS opt_tema
                     | QUIERO CURSOS opt_tema
                     | QUIERO INFORMACION opt_tema'''
        if len(p) == 6:
            topic = p[5]
        elif len(p) == 4:
            topic = p[3]
        else:
            topic = None
        self.last_sem = {'intent': 'SOLICITUD', 'topic': topic}

    def p_opt_info(self, p):
        '''opt_info : INFORMACION
                    | empty'''
        pass

    def p_opt_sobre(self, p):
        '''opt_sobre : SOBRE
                     | empty'''
        pass

    def p_opt_tema(self, p):
        '''opt_tema : empty
                    | SOBRE tema'''
        p[0] = p[2] if len(p) == 3 else None

    def p_tema(self, p):
        'tema : TOPIC'
        p[0] = p[1]

    def p_empty(self, p):
        'empty :'
        pass

    def p_error(self, p):
        # Consumir hasta EOL para seguir
        if p is None: 
            return
        tok = p
        while tok and tok.type != 'EOL':
            tok = parser.token()
        self.last_sem = {'intent': 'ERROR'}

_parser = ChatParser()
parser  = yacc.yacc(module=_parser, start='input', write_tables=False, debug=False)

#   Helpers pedidos  
def tokenize(text: str):
    """Devuelve [(TYPE, value), ...] tokenizando con el lexer PLY del chatbot."""
    inst = lex.lex(module=_lex, reflags=re.IGNORECASE | re.UNICODE)  # lexer fresco
    inst.input(text)
    out = []
    while True:
        tok = inst.token()
        if not tok:
            break
        out.append((tok.type, tok.value))
    return out

def parse_and_sem(text: str):
    """Parsea texto completo y devuelve la semántica (intent/topic)."""
    parser.parse(text, lexer=lexer)
    return _parser.last_sem

# Pruebas
print("PRUEBA:")
print("Tokens:", tokenize("hola\n"))
print("Sem   :", parse_and_sem("hola\n"))
print("Tokens:", tokenize("que hora es\n"))
print("Sem   :", parse_and_sem("que hora es\n"))
print("Tokens:", tokenize("quiero informacion sobre cursos de deep learning\n"))
print("Sem   :", parse_and_sem("quiero informacion sobre cursos de deep learning\n"))

# Respuestas por REGLAS
# Usa el lexer PLY para obtener "palabras" compatibles con tus reglas.

def _tokens_for_rules(text: str):
    """Convierte el texto en una lista de 'palabras' para el clasificador por reglas.
       Aprovecha el lexer PLY ya definido arriba (_lex).
    """
    inst = _lexmod.lex(module=_lex, reflags=re.IGNORECASE | re.UNICODE)
    # Asegurar EOL para que el lexer/emparejador se comporte como en modo línea
    if not text.endswith("\n"):
        text = text + "\n"
    inst.input(text)

    words = []
    while True:
        tok = inst.token()
        if not tok:
            break
        # Expandimos tokens multi-palabra a palabras sueltas para tus reglas
        if tok.type == "QUEHORA":
            words += ["que", "hora", "es"]
        elif tok.type in {"HOLA", "QUIERO", "INFORMACION", "SOBRE", "CURSOS", "TOPIC"}:
            # 'value' puede venir como 'hola' o 'que_hora_es'; normalizamos split por '_' y espacios
            words += tok.value.replace("_", " ").lower().split()
        # Ignoramos EOL/puntuación/otros
    return words

def classify_intent(tokens):
    # Gramática mínima implícita por patrones exactos
    if tokens == ['hola'] or tokens == ['buenos', 'dias']:
        return 'SALUDO'
    if tokens == ['que', 'hora', 'es']:
        return 'PREGUNTA_HORA'
    if tokens == ['quiero', 'informacion', 'sobre', 'cursos']:
        return 'SOLICITUD_INFO'
    return 'DESCONOCIDO'

def semantic_action(intent):
    if intent == 'SALUDO':
        return "¡Hola! ¿Cómo estás?"
    if intent == 'PREGUNTA_HORA':
        import datetime
        return f"La hora actual es {datetime.datetime.now().strftime('%H:%M:%S')}"
    if intent == 'SOLICITUD_INFO':
        return "Tenemos cursos de Machine Learning, Simulación y Redes. ¿Cuál te interesa?"
    return "No entendí tu consulta. ¿Podés reformularla?"

def chatbot_once(text: str):
    """Primero intenta con la CFG de PLY; si no clasifica, cae al clasificador por reglas."""
    # 1) Parser CFG
    parser.parse(text if text.endswith("\n") else text + "\n", lexer=lexer)
    sem = _parser.last_sem
    if sem and sem.get("intent") in {"SALUDO", "PREGUNTA", "SOLICITUD"}:
        if sem["intent"] == "SALUDO":
            return "¡Hola! ¿Cómo estás?"
        if sem["intent"] == "PREGUNTA":
            import datetime
            return f"La hora actual es {datetime.datetime.now().strftime('%H:%M:%S')}"
        if sem["intent"] == "SOLICITUD":
            return "Tenemos cursos de Machine Learning, Simulación y Redes. ¿Cuál te interesa?"

    # 2) Fallback por reglas
    toks = _tokens_for_rules(text)
    intent = classify_intent(toks)
    return semantic_action(intent)

#   Pruebas rápidas integradas  
print("\nDEMOS (fallback por reglas si la CFG no matchea):")
for u in ["hola", "que hora es", "quiero informacion sobre cursos", "adios"]:
    print(f"Usuario: {u}\nBot:     {chatbot_once(u)}\n")

# SVO + Semántica
# SVO = SUJETO+VERBO+OBJETO
# Parser SVO por reglas simples (sin PLY) para testear errores semánticos.
# Formato esperado (simplificado): ART SUST VERBO ART/AL SUST
# Ej.: "El perro muerde al perro"  (válido: ANIMAL muerde ANIMAL)
#     "La idea muerde al perro"   (inválido: ABSTRACTO no puede morder)

import re
from unidecode import unidecode  # pip install unidecode

#   Léxico / normalización  
def tokenize_svo(text: str):
    s = unidecode(text.strip().lower())
    # normalizamos contracción 'al' -> 'el'
    s = re.sub(r"\bal\b", "el", s)
    # quitamos puntuación simple
    s = re.sub(r"[,\.\!\?\;\:\(\)\[\]\{\}]+", " ", s)
    toks = s.split()
    return toks

#   Modelo semántico mínimo  
ENT = {}    # mapea (art, sust) -> id
TYPES = {}  # mapea id -> tipo ('ANIMAL', 'ABSTRACTO', ...)

# Diccionario de selección semántica para verbos
# 'muerde': sujeto ANIMAL, objeto ANIMAL
VERB_SIG = {
    "muerde": {"subj": "ANIMAL", "obj": "ANIMAL"},
    # Podés agregar más verbos acá
}

# Poblamos un par de entidades por defecto
ENT[("el", "perro")] = "perro1";   TYPES["perro1"] = "ANIMAL"
ENT[("el", "gato")]  = "gato1";    TYPES["gato1"]  = "ANIMAL"

#   Parser por patrón (no PLY) + chequeo semántico  
def parse_and_sem_svo(tokens):
    """
    tokens: lista de strings (p. ej., ['la','idea','muerde','el','perro'])
    Devuelve dict con semántica o lanza ValueError en caso de error semántico.
    """
    if len(tokens) < 5:
        raise ValueError(f"Estructura incompleta: {tokens}")

    # Esperamos: ART SUST VERBO ART SUST
    art1, sust1, verbo, art2, sust2 = tokens[:5]

    # Resolver entidades
    key1 = (art1, sust1)
    key2 = (art2, sust2)

    if key1 not in ENT:
        raise ValueError(f"Sujeto desconocido: {key1}")
    if key2 not in ENT:
        raise ValueError(f"Objeto desconocido: {key2}")

    subj_id = ENT[key1]; subj_ty = TYPES.get(subj_id, "DESCONOCIDO")
    obj_id  = ENT[key2]; obj_ty  = TYPES.get(obj_id,  "DESCONOCIDO")

    # Selección semántica del verbo
    sig = VERB_SIG.get(verbo)
    if not sig:
        raise ValueError(f"Verbo sin firma semántica definida: '{verbo}'")

    need_subj = sig.get("subj")
    need_obj  = sig.get("obj")

    if need_subj and subj_ty != need_subj:
        raise ValueError(f"Tipo de SUJETO inválido para '{verbo}': se requiere {need_subj}, llegó {subj_ty}")
    if need_obj and obj_ty != need_obj:
        raise ValueError(f"Tipo de OBJETO inválido para '{verbo}': se requiere {need_obj}, llegó {obj_ty}")

    # Si todo OK, devolvemos una estructura semántica
    return {
        "pred": verbo,
        "subj": {"id": subj_id, "type": subj_ty, "form": f"{art1} {sust1}"},
        "obj":  {"id": obj_id,  "type": obj_ty,  "form": f"{art2} {sust2}"},
    }

   
# Agregamos una entidad no animal y su tipo ABSTRACTO
ENT[("la", "idea")] = "idea1"
TYPES["idea1"] = "ABSTRACTO"

# Caso que debe FALLAR semánticamente:
try:
    bad_tokens = tokenize_svo("La idea muerde al perro")
    print("Tokens (bad):", bad_tokens)
    print(parse_and_sem_svo(bad_tokens))
except Exception as e:
    print("Error semántico detectado:", e)

# Caso que debe PASAR:
try:
    ok_tokens = tokenize_svo("El perro muerde al perro")
    print("Tokens (ok):", ok_tokens)
    print(parse_and_sem_svo(ok_tokens))
except Exception as e:
    print("Error inesperado:", e)




Overwriting chatbotCFG_ply.py


## 5. Parser con NLTK y construcción del árbol sintáctico
- Ejecutar el siguiente bloque para construir un `ChartParser` con una gramática mínima y obtener árboles de derivación.


In [1]:
# Ejecutar este bloque sólo si NLTK disponible en entorno.
try:
    import nltk
    grammar   nltk.CFG.fromstring("""
    S -> NP VP
    NP -> Det N | N
    VP -> V NP_a
    NP_a -> 'a' NP
    Det -> 'el'
    N -> 'gato' | 'perro'
    V -> 'muerde'
    """)
    parser   nltk.ChartParser(grammar)
    sent   ['el','gato','muerde','a','el','perro']
    print("Árboles de parse para:", sent)
    for tree in parser.parse(sent):
        print(tree)
except ModuleNotFoundError as e:
    print("NLTK no está instalado en este entorno. Instalarlo.")

Árboles de parse para: ['el', 'gato', 'muerde', 'a', 'el', 'perro']
(S
  (NP (Det el) (N gato))
  (VP (V muerde) (NP_a a (NP (Det el) (N perro)))))


## 6. Clasificador de intents con una red neuronal simple usando scikit-learn 

### NOTA: ESTE TEMA ESTA FUERA DEL ALCANCE DE LA MATERIA. SE PRESENTA EL EJEMPLO A MODO DE PRUEBA.

#### Ejemplo 1 

- Arma un dataset ejemplo (SALUDO / PREGUNTA_HORA / SOLICITUD), normaliza el texto (minúsculas + sin acentos), vectoriza con TF-IDF n-grams y entrena un MLP (una capa oculta).
- Al final evalúa y prueba con frases nuevas.
- Normalización: lowercase + quitar acentos.
- Vectorización: TF-IDF de 1-2 grams (captura “que hora”).
- Red: MLPClassifier con una capa oculta de 64 neuronas (ReLU + Adam).
- Clases: SALUDO, PREGUNTA_HORA, SOLICITUD.

#### Ejemplo 2
- Detección/corrección léxica: compara cada token contra el vocabulario de entrenamiento y propone la mejor coincidencia ( umbral cutoff 0.83).
- Ej.: “informaxion” → “informacion”.
- Si no encuentra palabra, lo marca como desconocido pero igual intenta clasificar.
- Diagnóstico en consola: muestra texto normalizado, tokens corregidos, OOV, intent + confianza y top-3.
- UI (ipywidgets): campo de texto + botón. Imprime diagnóstico y respuesta.
- Respuestas: SALUDO, PREGUNTA_HORA (con hora del sistema), SOLICITUD; si no entiende, devuelve una respuesta amable.


In [2]:
# -*- coding: utf-8 -*-
# Red neuronal simple (MLP) para intents: SALUDO, PREGUNTA_HORA, SOLICITUD

from unidecode import unidecode
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import numpy as np
import random

  
# 1) Dataset mini (ejemplo)
  
saludos = [
    "hola", "buenas", "buenos dias", "buenas tardes", "buenas noches",
    "hola como estas", "hola que tal", "hola, buen dia"
]
preg_hora = [
    "que hora es", "me decis la hora", "tenes la hora", "tienes la hora",
    "me podrias decir la hora", "que hora es ahora", "la hora por favor"
]
solicitudes = [
    "quiero informacion sobre cursos", "quisiera cursos de redes",
    "me gustaria informacion de cursos", "quiero cursos sobre machine learning",
    "necesito informacion sobre capacitaciones", "informacion de formacion por favor",
    "informacion sobre cursos de simulacion", "quiero curso de deep learning"
]

X = saludos + preg_hora + solicitudes
y = (["SALUDO"] * len(saludos)
     + ["PREGUNTA_HORA"] * len(preg_hora)
     + ["SOLICITUD"] * len(solicitudes))

  
# 2) Normalización simple
  
def normalize(s: str) -> str:
    # minúsculas + sin acentos + colapsar espacios
    s = unidecode(s.lower())
    s = " ".join(s.split())
    return s

X_norm = [normalize(t) for t in X]

  
# 3) Split train/test
  
X_train, X_test, y_train, y_test = train_test_split(
    X_norm, y, test_size=0.3, random_state=42, stratify=y
)

  
# 4) Pipeline TF-IDF + MLP
  
# n-grams (1 a 2) suelen ayudar en frases cortas
pipe = Pipeline([
    ("tfidf", TfidfVectorizer(ngram_range=(1,2), min_df=1)),
    ("mlp", MLPClassifier(
        hidden_layer_sizes=(64,),  # una capa oculta de 64 neuronas
        activation="relu",
        solver="adam",
        alpha=1e-3,
        batch_size=8,
        learning_rate_init=1e-3,
        max_iter=300,
        random_state=42
    ))
])

pipe.fit(X_train, y_train)

  
# 5) Evaluación
  
y_pred = pipe.predict(X_test)
print("== Reporte en test ==")
print(classification_report(y_test, y_pred, digits=3))

  
# 6) Pruebas manuales
  
pruebas = [
    "Hola!", 
    "Me decís la hora por favor?",
    "Quisiera información sobre cursos de deep learning",
    "Buenas noches",
    "La hora ahora",
    "Necesito capacitaciones en simulacion",
]
pruebas_norm = [normalize(p) for p in pruebas]
preds = pipe.predict(pruebas_norm)
proba = pipe.predict_proba(pruebas_norm)
labels = pipe.classes_

print("\n== Predicciones ==")
for txt, pred, pr in zip(pruebas, preds, proba):
    top = sorted(list(zip(labels, pr)), key=lambda x: x[1], reverse=True)[:3]
    conf = ", ".join([f"{lbl}:{p:.2f}" for lbl, p in top])
    print(f"- '{txt}' -> {pred}  [{conf}]")


== Reporte en test ==
               precision    recall  f1-score   support

PREGUNTA_HORA      1.000     1.000     1.000         2
       SALUDO      1.000     1.000     1.000         3
    SOLICITUD      1.000     1.000     1.000         2

     accuracy                          1.000         7
    macro avg      1.000     1.000     1.000         7
 weighted avg      1.000     1.000     1.000         7


== Predicciones ==
- 'Hola!' -> SALUDO  [SALUDO:0.97, SOLICITUD:0.02, PREGUNTA_HORA:0.01]
- 'Me decís la hora por favor?' -> PREGUNTA_HORA  [PREGUNTA_HORA:1.00, SOLICITUD:0.00, SALUDO:0.00]
- 'Quisiera información sobre cursos de deep learning' -> SOLICITUD  [SOLICITUD:1.00, SALUDO:0.00, PREGUNTA_HORA:0.00]
- 'Buenas noches' -> SALUDO  [SALUDO:0.99, PREGUNTA_HORA:0.00, SOLICITUD:0.00]
- 'La hora ahora' -> PREGUNTA_HORA  [PREGUNTA_HORA:0.98, SALUDO:0.01, SOLICITUD:0.01]
- 'Necesito capacitaciones en simulacion' -> SOLICITUD  [SOLICITUD:0.79, SALUDO:0.14, PREGUNTA_HORA:0.07]


#### Segundo ejemplo

In [7]:
# -*- coding: utf-8 -*-
# Clasificador de intents con MLP + manejo de "errores léxicos" (corrección simple)
# Requiere: scikit-learn, unidecode. (UI opcional: ipywidgets, pandas)

from unidecode import unidecode
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import numpy as np
import difflib
import datetime

#   1) Dataset mini   
saludos = [
    "hola", "buenas", "buenos dias", "buenas tardes", "buenas noches",
    "hola como estas", "hola que tal", "hola, buen dia"
]
preg_hora = [
    "que hora es", "me decis la hora", "tenes la hora", "tienes la hora",
    "me podrias decir la hora", "que hora es ahora", "la hora por favor"
]
solicitudes = [
    "quiero informacion sobre cursos", "quisiera cursos de redes",
    "me gustaria informacion de cursos", "quiero cursos sobre machine learning",
    "necesito informacion sobre capacitaciones", "informacion de formacion por favor",
    "informacion sobre cursos de simulacion", "quiero curso de deep learning"
]

X = saludos + preg_hora + solicitudes
y = (["SALUDO"] * len(saludos)
     + ["PREGUNTA_HORA"] * len(preg_hora)
     + ["SOLICITUD"] * len(solicitudes))

#   2) Normalización & tokenización   
def normalize(s: str) -> str:
    s = unidecode(s.lower())
    s = " ".join(s.split())
    return s

def tokenize(s: str):
    return normalize(s).split()

X_norm = [normalize(t) for t in X]

#   3) Split   
X_train, X_test, y_train, y_test = train_test_split(
    X_norm, y, test_size=0.3, random_state=42, stratify=y
)

#  4) Pipeline TF-IDF + MLP   
pipe = Pipeline([
    ("tfidf", TfidfVectorizer(ngram_range=(1,2), min_df=1)),
    ("mlp", MLPClassifier(
        hidden_layer_sizes=(64,),
        activation="relu",
        solver="adam",
        alpha=1e-3,
        batch_size=8,
        learning_rate_init=1e-3,
        max_iter=300,
        random_state=42
    ))
])
pipe.fit(X_train, y_train)

#  5) Evaluación rápida   
y_pred = pipe.predict(X_test)
print("== Reporte en test ==")
print(classification_report(y_test, y_pred, digits=3))

#  6) Vocabulario para detectar/corregir OOV   
# Construimos un vocabulario "léxico" básico desde el set de tokens de entrenamiento
lex_vocab = set()
for s in X_norm:
    lex_vocab.update(s.split())

def detect_and_correct_tokens(text: str, vocab: set[str], cutoff: float = 0.83):
    """
    Devuelve:
      corrected_tokens: lista de tokens (posiblemente corregidos)
      oov_report: lista de dicts con token original, sugerencia y confianza aproximada
    """
    toks = tokenize(text)
    corrected = []
    report = []
    for t in toks:
        if t in vocab:
            corrected.append(t)
        else:
            # Proponemos la mejor coincidencia del vocabulario (difflib)
            cand = difflib.get_close_matches(t, list(vocab), n=1, cutoff=cutoff)
            if cand:
                corrected.append(cand[0])
                report.append({"token": t, "suggestion": cand[0], "note": "corregido"})
            else:
                corrected.append(t)  # sin corrección
                report.append({"token": t, "suggestion": None, "note": "desconocido"})
    return corrected, report

#  7) Predicción + respuesta (aunque haya errores léxicos)   
def predict_intent_with_lex(text: str):
    """
    - Normaliza, detecta OOV y corrige.
    - Predice con el texto corregido.
    - Devuelve intent, probas y reporte de OOV.
    """
    corrected_tokens, oov = detect_and_correct_tokens(text, lex_vocab)
    corrected_text = " ".join(corrected_tokens)
    proba = pipe.predict_proba([corrected_text])[0]
    labels = pipe.classes_
    top_idx = int(np.argmax(proba))
    intent = labels[top_idx]
    conf = float(proba[top_idx])
    top3 = sorted(list(zip(labels, proba)), key=lambda x: x[1], reverse=True)[:3]
    return {
        "original_text": text,
        "normalized": normalize(text),
        "corrected_text": corrected_text,
        "oov": oov,
        "intent": intent,
        "confidence": conf,
        "top3": top3
    }

def generate_reply(intent: str) -> str:
    if intent == "SALUDO":
        return "¡Hola! ¿Cómo estás?"
    if intent == "PREGUNTA_HORA":
        # Hora local del sistema
        return f"Son las {datetime.datetime.now().strftime('%H:%M:%S')}."
    if intent == "SOLICITUD":
        return "Tenemos cursos de Machine Learning, Simulación y Redes. ¿Cuál te interesa?"
    # fallback
    return "No entendí bien tu consulta, pero puedo ayudarte con saludos, la hora o información de cursos."

def chat_once(text: str, show_debug: bool = True):
    info = predict_intent_with_lex(text)
    reply = generate_reply(info["intent"])
    if show_debug:
        print("DETECCION ERRORES:")
        print("original   :", info["original_text"])
        print("normalized :", info["normalized"])
        print("corrected  :", info["corrected_text"])
        if info["oov"]:
            print("lex-issues :",
                  [f"{r['token']}→{r['suggestion'] or '∅'}({r['note']})" for r in info["oov"]])
        print("intent     :", info["intent"], f"(conf={info['confidence']:.2f})")
        print("top3       :", ", ".join([f"{l}:{p:.2f}" for l,p in info["top3"]]))
        print("    ==")
    return reply

#  8) Pruebas manuales (incluye errores léxicos)   
tests = [
    "Hola!",
    "Me decis la hora porfas?",         # 'porfas' no está; se intentará corregir
    "Quisiera informaxion sobre cursos",# 'informaxion' error
    "Bunas nochess",                    # errores múltiples
    "Kiero cursoz sobr redes",          # leetspeak-ish
    "La hora ahora",
    "Necesito capacitaciones en simulacion",
]
print("\n== Predicciones y respuestas (con detección/corrección lexical) ==")
for t in tests:
    print(f"- Usuario: {t}")
    print(f"  Bot   : {chat_once(t, show_debug=True)}\n")

#  9) UI interactiva con ipywidgets (si está disponible)   
try:
    import ipywidgets as widgets
    from IPython.display import display, clear_output, HTML

    inp = widgets.Text(
        value="Hlaaa bnas nochess, me decis la hroa?",
        description="Tu mensaje:",
        layout=widgets.Layout(width="100%")
    )
    btn = widgets.Button(description="Enviar", button_style="primary")
    out = widgets.Output()

    def on_click(_):
        with out:
            clear_output()
            resp = chat_once(inp.value, show_debug=True)
            print("\nRespuesta:", resp)

    btn.on_click(on_click)
    box = widgets.VBox([inp, btn, out])
    display(HTML("<h3>Chat — MLP + Corrección léxica</h3>"))
    display(box)
except Exception:
    print("\n[UI] ipywidgets no disponible. Usá chat_once('...') en celdas o hacé un loop input().")


== Reporte en test ==
               precision    recall  f1-score   support

PREGUNTA_HORA      1.000     1.000     1.000         2
       SALUDO      1.000     1.000     1.000         3
    SOLICITUD      1.000     1.000     1.000         2

     accuracy                          1.000         7
    macro avg      1.000     1.000     1.000         7
 weighted avg      1.000     1.000     1.000         7


== Predicciones y respuestas (con detección/corrección lexical) ==
- Usuario: Hola!
DETECCION ERRORES:
original   : Hola!
normalized : hola!
corrected  : hola
lex-issues : ['hola!→hola(corregido)']
intent     : SALUDO (conf=0.97)
top3       : SALUDO:0.97, SOLICITUD:0.02, PREGUNTA_HORA:0.01
    ==
  Bot   : ¡Hola! ¿Cómo estás?

- Usuario: Me decis la hora porfas?
DETECCION ERRORES:
original   : Me decis la hora porfas?
normalized : me decis la hora porfas?
corrected  : me decis la hora porfas?
lex-issues : ['porfas?→∅(desconocido)']
intent     : PREGUNTA_HORA (conf=0.99)
top3       :

## 8. Ejercicios propuestos
- Extender el chatbot con m{as frases y tokens.
- Hacer el chatbot interactivo desde la terminal.
- Analizar errores semánticos del chatbot
- Extender el léxico agregando `('el','raton') → raton1` y probando frases:
   - "el perro muerde a el raton"
   - "el raton muerde a el perro"
- Agregar un segundo verbo `persigue` con semántica `PERSIGUE(subj, obj)` y mostrar:
   - `PERSIGUE(perro1, gato1)`
- Control de tipos: forzar un error semántico con un sujeto no animal e interpretar el mensaje.
- Chatbot: agregar una intención `AYUDA` para frases como *"ayuda"*, *"necesito ayuda"* y responder con un menú.


## 9. Referencias y notas
- Aho, Lam, Sethi, Ullman. **Compilers: Principles, Techniques, and Tools**.
- Jurafsky & Martin. **Speech and Language Processing**.
- Hopcroft, Motwani, Ullman. **Introduction to Automata Theory, Languages, and Computation**.
- Notas de clase: **CFG, SDT
